In [1]:
from math import sin, cos, pi


class Polygon:
    def __init__(self, n_edges, r):

        self.edges = n_edges
        self.radius = r
        self.vertices = n_edges
        self.init_parameters()

    def init_parameters(self):
        _ = self.edge_length
        _ = self.interior_angle
        _ = self.apothem
        _ = self.area
        _ = self.perimeter

    @property
    def edges(self):
        return self._edges

    @property
    def radius(self):
        return self._radius

    @edges.setter
    def edges(self, n_edges):
        if not isinstance(n_edges, (int, float)):
            raise ValueError
        if n_edges < 3:
            raise ValueError

        self._edges = n_edges
        self._vertices = n_edges
        self._edge_length = None
        self._interior_angle = None
        self._apothem = None
        self._area = None
        self._perimeter = None

    @property
    def vertices(self):
        return self._vertices

    @vertices.setter
    def vertices(self, vert):
        if not isinstance(vert, (int, float)):
            raise ValueError
        if vert < 3:
            raise ValueError

        self._edges = vert
        self._vertices = vert
        self._edge_length = None
        self._interior_angle = None
        self._apothem = None
        self._area = None
        self._perimeter = None

    @radius.setter
    def radius(self, r):
        if not isinstance(r, (int, float)):
            raise ValueError
        if r < 0:
            raise ValueError
        self._radius = r
        self._edge_length = None
        self._interior_angle = None
        self._apothem = None
        self._area = None
        self._perimeter = None

    @property
    def edge_length(self):
        if self._edge_length is None:
            self._edge_length = 2 * self._radius * sin(pi / self._edges)

        return self._edge_length

    @property
    def interior_angle(self):
        if self._interior_angle is None:
            self._interior_angle = (self._edges - 2) * 180 / self._edges
        return self._interior_angle

    @property
    def apothem(self):
        if self._apothem is None:
            self._apothem = self._radius * cos(pi / self._edges)
        return self._apothem

    @property
    def area(self):
        if self._apothem is None:
            _ = self.apothem
        if self._edge_length is None:
            _ = self.edge_length
        if self._area is None:
            self._area = 0.5 * self._edges * self._edge_length * self._apothem
        return self._area

    @property
    def perimeter(self):
        if self._edge_length is None:
            _ = self.edge_length
        if self._perimeter is None:
            self._perimeter = self._edges * self._edge_length
        return self._perimeter

    def __repr__(self):
        return f"A regular polygon with {vars(self)}"

    def __eq__(self, other):
        return (self.vertices == other.vertices) and (self.radius == other.radius)

    def __gt__(self, other):
        return self.vertices > other.vertices


In [5]:
from functools import lru_cache
class PolygonSequence:
    def __init__(self, max_edge, radius):
        self.max_edge = max_edge
        self.radius = radius
        self.ratio = {}

    def init_parameters(self):
        _ = self.max_edge
        _ = self.radius

    @property
    def max_edge(self):
        return self._max_edge

    @max_edge.setter
    def max_edge(self, new_max_edge):
        if not isinstance(new_max_edge, (int, float)):
            raise ValueError
        if new_max_edge < 3:
            raise ValueError

        self._max_edge = new_max_edge

    @property
    def radius(self):

        return self._radius

    @radius.setter
    def radius(self, rad):
        if not isinstance(rad, (int, float)):
            raise ValueError
        if rad < 0:
            raise ValueError

        self._radius = rad

    def __repr__(self):
        return f"Polygon Sequence class with max_edge = {self.max_edge} and common radius = {self.radius}"

    def __len__(self):
        return self.max_edge - 2

    def __getitem__(self, idx):
        if isinstance(idx, int):
            if idx >= 0:
                if idx < 3:
                    idx = 3
                return PolygonSequence.get_poly(idx, self.radius)

            else:
                idx = self.max_edge + idx
                if idx < 0 or idx >= self.max_edge:
                    raise IndexError
                return PolygonSequence.get_poly(idx + 1, self.radius)
        else:
            start, end, skip = idx.indices(self.max_edge + 1)
            start = start if start > 3 else 3
            end = end if end >= 3 else 2
            return [
                PolygonSequence.get_poly(i, self.radius)
                for i in range(start, end, skip)
            ]

    @staticmethod
    @lru_cache(16)
    def get_poly(n, radius):
        return Polygon(n, radius)

    @property
    @lru_cache(1)
    def max_efficiency(self):
        for i in range(3, self.max_edge + 1):
            poly = self.__getitem__(i)
            self.ratio[i] = poly.area * 1.0 / poly.perimeter
        max_key = max(self.ratio, key=self.ratio.get)
        # return f"max_efficiency_polygon_edge: {max_key}, area/perimeter ratio : {self.ratio[max_key]}"
        return self.get_poly(max_key, self.radius)

    def __iter__(self):
        return self.PolygonIter(self)

    class PolygonIter:
        def __init__(self, polygon_object):
            self._polygon_object = polygon_object
            self._idx = 3

        def __iter__(self):
            return self

        def __next__(self):
            if self._idx > self._polygon_object.max_edge:
                raise StopIteration
            else:
                poly = Polygon(self._idx, self._polygon_object._radius)
                self._idx += 1
                return poly



In [9]:
import math
def test_polygon():
    abs_tol = 0.001
    rel_tol = 0.001
    
    try:
        p = Polygon(2, 10)
        assert False, ('Creating a Polygon with 2 sides: '
                       ' Exception expected, not received')
    except ValueError:
        pass
                       
    n = 3
    R = 1
    p = Polygon(n, R)
    assert p.vertices == n, (f'actual: {p.vertices},'
                                   f' expected: {n}')
    assert p.edges == n, f'actual: {p.count_edges}, expected: {n}'
    assert p.radius == R, f'actual: {p.circumradius}, expected: {n}'
    assert p.interior_angle == 60, (f'actual: {p.interior_angle},'
                                    ' expected: 60')
    n = 4
    R = 1
    p = Polygon(n, R)
    assert p.interior_angle == 90, (f'actual: {p.interior_angle}, '
                                    ' expected: 90')

    
    assert math.isclose(p.edge_length, math.sqrt(2),
                       rel_tol=rel_tol,
                       abs_tol=abs_tol), (f'actual: {p.edge_length},'
                                          f' expected: {math.sqrt(2)}')
    
    assert math.isclose(p.perimeter, 4 * math.sqrt(2),
                       rel_tol=rel_tol,
                       abs_tol=abs_tol), (f'actual: {p.perimeter},'
                                          f' expected: {4 * math.sqrt(2)}')
    
    assert math.isclose(p.apothem, 0.707,
                       rel_tol=rel_tol,
                       abs_tol=abs_tol), (f'actual: {p.perimeter},'
                                          ' expected: 0.707')
    assert math.isclose(p.area, 2, 
                    rel_tol=abs_tol, 
                    abs_tol=abs_tol), (f'actual: {p.area},'
                                        ' expected: 2.0')
    p = Polygon(6, 2)
    assert math.isclose(p.edge_length, 2,
                        rel_tol=rel_tol, abs_tol=abs_tol)
    assert math.isclose(p.apothem, 1.73205,
                        rel_tol=rel_tol, abs_tol=abs_tol)
    assert math.isclose(p.area, 10.3923,
                        rel_tol=rel_tol, abs_tol=abs_tol)
    assert math.isclose(p.perimeter, 12,
                        rel_tol=rel_tol, abs_tol=abs_tol)
    assert math.isclose(p.interior_angle, 120,
                        rel_tol=rel_tol, abs_tol=abs_tol)
    
    p = Polygon(12, 3)
    assert math.isclose(p.edge_length, 1.55291,
                        rel_tol=rel_tol, abs_tol=abs_tol)
    assert math.isclose(p.apothem, 2.89778,
                        rel_tol=rel_tol, abs_tol=abs_tol)
    assert math.isclose(p.area, 27,
                        rel_tol=rel_tol, abs_tol=abs_tol)
    assert math.isclose(p.perimeter, 18.635,
                        rel_tol=rel_tol, abs_tol=abs_tol)
    assert math.isclose(p.interior_angle, 150,
                        rel_tol=rel_tol, abs_tol=abs_tol)
    
    p1 = Polygon(3, 10)
    p2 = Polygon(10, 10)
    p3 = Polygon(15, 10)
    p4 = Polygon(15, 100)
    p5 = Polygon(15, 100)
    
    assert p2 > p1
    assert p2 < p3
    assert p3 != p4
    assert p1 != p4
    assert p4 == p5

test_polygon()

In [11]:
sequences = PolygonSequence(20,10)


The `PolygonSequence` class has an `__init__`, but no `__next__`

In [12]:
'__init__' in dir(sequences), '__next__' in dir(sequences)


(True, False)

The `SequenceIterator` class has both `__init__` and `__next__` methods

In [13]:
'__init__' in dir(iter(sequences)), '__next__' in dir(iter(sequences))


(True, True)

In [14]:
# The PolygonSequence can be iterated over
for seq in sequences:
    print(seq)

A regular polygon with {'_edges': 3, '_vertices': 3, '_edge_length': 17.32050807568877, '_interior_angle': 60.0, '_apothem': 5.000000000000001, '_area': 129.9038105676658, '_perimeter': 51.96152422706631, '_radius': 10}
A regular polygon with {'_edges': 4, '_vertices': 4, '_edge_length': 14.14213562373095, '_interior_angle': 90.0, '_apothem': 7.0710678118654755, '_area': 200.0, '_perimeter': 56.5685424949238, '_radius': 10}
A regular polygon with {'_edges': 5, '_vertices': 5, '_edge_length': 11.755705045849464, '_interior_angle': 108.0, '_apothem': 8.090169943749475, '_area': 237.76412907378844, '_perimeter': 58.77852522924732, '_radius': 10}
A regular polygon with {'_edges': 6, '_vertices': 6, '_edge_length': 9.999999999999998, '_interior_angle': 120.0, '_apothem': 8.660254037844387, '_area': 259.80762113533154, '_perimeter': 59.999999999999986, '_radius': 10}
A regular polygon with {'_edges': 7, '_vertices': 7, '_edge_length': 8.677674782351163, '_interior_angle': 128.57142857142858,

In [15]:
# The PolygonIterator class can be iterated over using the `next` method
sequences = iter(PolygonSequence(25,10))
for i in range(5):
    print(next(sequences))

A regular polygon with {'_edges': 3, '_vertices': 3, '_edge_length': 17.32050807568877, '_interior_angle': 60.0, '_apothem': 5.000000000000001, '_area': 129.9038105676658, '_perimeter': 51.96152422706631, '_radius': 10}
A regular polygon with {'_edges': 4, '_vertices': 4, '_edge_length': 14.14213562373095, '_interior_angle': 90.0, '_apothem': 7.0710678118654755, '_area': 200.0, '_perimeter': 56.5685424949238, '_radius': 10}
A regular polygon with {'_edges': 5, '_vertices': 5, '_edge_length': 11.755705045849464, '_interior_angle': 108.0, '_apothem': 8.090169943749475, '_area': 237.76412907378844, '_perimeter': 58.77852522924732, '_radius': 10}
A regular polygon with {'_edges': 6, '_vertices': 6, '_edge_length': 9.999999999999998, '_interior_angle': 120.0, '_apothem': 8.660254037844387, '_area': 259.80762113533154, '_perimeter': 59.999999999999986, '_radius': 10}
A regular polygon with {'_edges': 7, '_vertices': 7, '_edge_length': 8.677674782351163, '_interior_angle': 128.57142857142858,

In [20]:
# The Polygon Sequence class can be iterated over using index too, as __getitem__ method has been implemented
sequences = PolygonSequence(25,10)
for i in range(3,6):
    print(sequences[i])

A regular polygon with {'_edges': 3, '_vertices': 3, '_edge_length': 17.32050807568877, '_interior_angle': 60.0, '_apothem': 5.000000000000001, '_area': 129.9038105676658, '_perimeter': 51.96152422706631, '_radius': 10}
A regular polygon with {'_edges': 4, '_vertices': 4, '_edge_length': 14.14213562373095, '_interior_angle': 90.0, '_apothem': 7.0710678118654755, '_area': 200.0, '_perimeter': 56.5685424949238, '_radius': 10}
A regular polygon with {'_edges': 5, '_vertices': 5, '_edge_length': 11.755705045849464, '_interior_angle': 108.0, '_apothem': 8.090169943749475, '_area': 237.76412907378844, '_perimeter': 58.77852522924732, '_radius': 10}


In [16]:
try:
    while True:
        print(next(sequences))
except StopIteration:
    print('Reached end of execution')

A regular polygon with {'_edges': 8, '_vertices': 8, '_edge_length': 7.653668647301796, '_interior_angle': 135.0, '_apothem': 9.238795325112868, '_area': 282.842712474619, '_perimeter': 61.22934917841437, '_radius': 10}
A regular polygon with {'_edges': 9, '_vertices': 9, '_edge_length': 6.840402866513374, '_interior_angle': 140.0, '_apothem': 9.396926207859085, '_area': 289.2544243589427, '_perimeter': 61.563625798620365, '_radius': 10}
A regular polygon with {'_edges': 10, '_vertices': 10, '_edge_length': 6.180339887498948, '_interior_angle': 144.0, '_apothem': 9.510565162951535, '_area': 293.8926261462365, '_perimeter': 61.80339887498948, '_radius': 10}
A regular polygon with {'_edges': 11, '_vertices': 11, '_edge_length': 5.634651136828594, '_interior_angle': 147.27272727272728, '_apothem': 9.594929736144973, '_area': 297.35244960057867, '_perimeter': 61.98116250511453, '_radius': 10}
A regular polygon with {'_edges': 12, '_vertices': 12, '_edge_length': 5.176380902050415, '_interi

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d546006b-e8c8-4509-90cd-8840fb739e0d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>